# Paris live traffic information

In [1]:
# !ls -lh traffic
# !rm data/aggregates.pd.h5 data/no_commas.h5

ls: cannot access 'traffic': No such file or directory
rm: cannot remove 'traffic/aggregates.pd.h5': No such file or directory
rm: cannot remove 'traffic/no_commas.h5': No such file or directory


In [3]:
from tqdm import tqdm_notebook as tqdm
import pandas as pd
import numpy as np

## Load map from counter id to geo location

Here we prepare a dictionary mapping from the counter id `id_arc_trafic` to its `lat,lon` position.
Since the db is tiny, we load everything in memory

In [4]:
from string import atof
def c_geo_point_2d(x):
    out = []
    for i in x.split(','):
        try:
            out.append(atof(i))
        except ValueError,msg:
            return [np.nan,np.nan]
            raise(ValueError(' x='+str(x)))
    return out
converters = {'geo_point_2d':c_geo_point_2d }

positions = pd.read_csv('data/referentiel-comptages-routiers.csv',delimiter=';',converters=converters)
positions['lat']=positions['geo_point_2d'].apply(lambda x:x[0])
positions['lon']=positions['geo_point_2d'].apply(lambda x:x[1])

from collections import defaultdict
posdict = defaultdict(lambda :{'lat':0,'lon':0})
for j,i in positions[['id_arc_tra','lat','lon']].iterrows():
    id_arc_tra = float(i.id_arc_tra)
    lat = i.lat
    lon = i.lon
    posdict[id_arc_tra]={'lat':lat,'lon':lon}
positions.head(5)

,OBJECTID,id_arc,id_arc_tra,SHAPE.LEN,geo_shape,geo_point_2d,lat,lon
0,1009,1106.0,4301.0,0.0,"{""type"": ""LineString"", ""coordinates"": [[2.2879...","[48.8804688997, 2.28668204502]",48.880469,2.286682
1,1013,1110.0,5868.0,0.0,"{""type"": ""LineString"", ""coordinates"": [[2.4033...","[48.8362023805, 2.4044924136]",48.836202,2.404492
2,1016,1114.0,242.0,0.0,"{""type"": ""LineString"", ""coordinates"": [[2.3237...","[48.8688115513, 2.32343707534]",48.868812,2.323437
3,1027,1125.0,1368.0,0.0,"{""type"": ""LineString"", ""coordinates"": [[2.3668...","[48.8639194501, 2.3663424412]",48.863919,2.366342
4,1028,1126.0,1366.0,0.0,"{""type"": ""LineString"", ""coordinates"": [[2.3658...","[48.8649639807, 2.36570360277]",48.864964,2.365704


## Prepare an `sql` db of traffic records
In contrast to an `hdf` format, an `sqlite` database offers optimized aggregation 
essentially for free.  Here we copy the original `csv` file to the db operating on chunks of 
5e4 items using the `chunk_size` paramers of pandas' `read_csv`. 
On the fly, we map from counter id `id_arc_trafic` to its gps position and precompute some columns. In particular the `BDay` columns, meaning `business day` is true when the day is mon->fri, false otherwise, holidays are not considered.
The automatic type detection algorithms fails for 3 columns leading to a `ValueError` when sending the rows to the `db`.  The type of these 3 lines is specified to be 16-bit int or floats, which is enough to store what we need.

In [5]:
print np.iinfo(np.int16)
print np.finfo(np.float16)

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------

Machine parameters for float16
---------------------------------------------------------------
precision=  3   resolution= 1.00040e-03
machep=   -10   eps=        9.76562e-04
negep =   -11   epsneg=     4.88281e-04
minexp=   -14   tiny=       6.10352e-05
maxexp=    16   max=        6.55040e+04
nexp  =     5   min=        -max
---------------------------------------------------------------



In [6]:
import sqlite3 as sql
conn = sql.connect('traffic.db')

In [7]:
in_it = pd.read_csv('traffic/no_commas.csv',delimiter=';',parse_dates=['horodate'],chunksize=50000,
                    dtype={'id_arc_trafic':np.int16, # this is NOT bcs I'm pedantic
                           'debit':np.float16,       # but bcs otherwise it won't infer the right columns types
                           'taux':np.float16})       # and will bail out with a misterious "ValueError"
from itertools import izip
for chunk in tqdm(in_it):
    try:
        # line by line pre-processing
        chunk.debit = chunk.debit.apply(lambda x: 0 if np.isnan(x) else x)
        chunk.taux = chunk.taux.apply(lambda x: 0 if np.isnan(x) else x)
        chunk=chunk.assign(hour=chunk.horodate.apply(lambda x:x.hour))
        chunk=chunk.assign(month=chunk.horodate.apply(lambda x:x.month))
        chunk=chunk.assign(BDay=chunk.horodate.apply(lambda x:x.isoweekday() not in (6,7)))
        # map from counter id to its position
        chunk = chunk.assign(lat=chunk.id_arc_trafic.apply(lambda x:posdict[float(x)]['lat']))
        chunk = chunk.assign(lon=chunk.id_arc_trafic.apply(lambda x:posdict[float(x)]['lon']))        
        chunk = chunk[['hour','month','BDay','debit','taux','id_arc_trafic','lat','lon']]        
        
        chunk.to_sql('traffic',conn,if_exists='append')
    except ValueError,msg:
        print msg
        print chunk
        break

rm: cannot remove 'traffic/no_commas.h5': No such file or directory


In [11]:
#safety checks
conn.commit()
cur = conn.cursor()
cur.execute('SELECT DISTINCT month FROM traffic')
cur.fetchall()

[(3,), (4,), (1,), (2,), (5,), (6,), (7,), (8,), (9,), (10,), (11,), (12,)]

In [12]:
cur.execute('select count(hour) from traffic')
cur.fetchall()[0]

(31583340,)

## Aggregate records
We keep only records of business days (when `BDay==True`) and group by hour and month, meaning that we are averaging over the
weekdays. The results of the query are read in chunks of 200 records and directly stored in a (this time small) hdf file using `pandas.HDFStore`. In this way the work is done mostly out-of-memory.

In [13]:
# aggregate by hour, month, and traffic counter (id_arc_trafic)
sql_query = 'SELECT hour,month,id_arc_trafic,AVG(debit),AVG(taux),lat,lon FROM traffic WHERE BDay GROUP BY hour, month, id_arc_trafic;'

In [14]:
!rm traffic/aggregates.hdf 
out = pd.HDFStore('traffic/aggregates.hdf')
for rows in tqdm(pd.read_sql_query(sql_query,conn,chunksize=200)):
    out.append(key = '/aggregates',value = rows)

In [15]:
agg = out['/aggregates']
out.close()

In [17]:
agg

,hour,month,id_arc_trafic,AVG(debit),AVG(taux),lat,lon
0,0,1,1,1188.714286,4.638207,48.859838,2.334242
1,0,1,2,1188.714286,0.000000,48.859375,2.336451
2,0,1,3,1188.714286,0.000000,48.859019,2.339782
3,0,1,4,1188.714286,0.000000,48.858747,2.341134
4,0,1,5,1479.809524,6.076404,48.858214,2.343447
5,0,1,6,1479.809524,13.699405,48.857746,2.345062
6,0,1,7,1479.809524,11.016927,48.857355,2.346448
7,0,1,8,1392.595238,8.957589,48.856938,2.348014
8,0,1,9,1364.214286,7.782878,48.856589,2.349191
9,0,1,10,1364.214286,7.782878,48.856269,2.350029
